# Final Project - image.fits

#### Here we are importing and setting up some modules

In [ ]:
import numpy as np
import sep

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10.,8.]

#### We are now opening and reading in the data from the fits file into a data variable

In [ ]:
hdul = fits.open("image.fits")
data = hdul[0].data

#### The following will display the image we are dealing with

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation = 'nearest', cmap = 'gray', vmin = m - s, vmax = m + s, origin = 'lower')
plt.colorbar();
plt.savefig("final_project_tutorial_figure_1.png")

#### We are using the sep background operation to measure the background of the image

In [ ]:
bkg = sep.Background(data)

#### The following values are the mean and noise of the background object that was returned previously

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

#### We will now interpret the background as an array like before with the image

In [ ]:
bkg_image = bkg.back()

#### The following will display the background

In [ ]:
plt.imshow(bkg_image, interpolation = 'nearest', cmap = 'gray', origin = 'lower')
plt.colorbar()
plt.savefig("final_project_tutorial_figure_2.png")

#### Now we will interpret the background noise as an array

In [ ]:
bkg_rms = bkg.rms()

#### The following will display the background noise

In [ ]:
plt.imshow(bkg_rms, interpolation = 'nearest', cmap = 'gray', origin = 'lower')
plt.colorbar()
plt.savefig("final_project_tutorial_figure_3.png")

#### We are now subtracting the background from the main image

In [ ]:
data_sub = data - bkg

#### The following function will detect objects in the image and store these objects in the objects variable

In [ ]:
objects = sep.extract(data_sub, 1.5, err = bkg.globalrms)

#### We then print out how many objects were detected

In [ ]:
len(objects)

#### The following will display the main image with the detected objects circled in red

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m, s, np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data, interpolation = 'nearest', cmap = 'gray', vmin = m - s, vmax = m + s, origin = 'lower')

for i in range(len(objects)):
    e = Ellipse(xy = (objects['x'][i], objects['y'][i]),
                width = 6 * objects['a'][i],
                height = 6 * objects['b'][i],
                angle = objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig("final_project_tutorial_figure_4.png")

#### The following displays the many fields that the objects data type has

In [ ]:
objects.dtype.names

#### We will now use some of these fields to perform some aperture photometry

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err = bkg.globalrms, gain = 1.0)

#### The following will display the first ten objects that fall under the guidelines we set above

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))